# <span style="color:mediumvioletred"> **Data Wrangling**</span>
### **Proyecto de Ciencia de Datos**
- Aissa Berenice Gonzalez Fosado 752224
- Clara Paola Aguilar Casillas
- Daniela de la Torre Gallo


<img src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal-Gris.png"  width="300">

In [ ]:
##  **Data Wrangling**
- Reglas aplicadas por tema: nulos, duplicados, outliers, casting/encoding, reglas de negocio.
- Mini-evidencias **antes/después** (tabla/figura).
- Estado del dataset limpio (ruta en `data/processed/` y esquema final).